# Demand analyses
Analyze the demand, both with and without demand response units in terms of
* Overall annual demand as well as
* Demand patterns

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Define data sets and global parameters

In [ ]:
# Model configuration in terms of prices and costs
demand_response_scenario = "50"

# inputs and outputs
path_folder = "./model_inputs/pommesinvest/"

file_names = {
    "demand_incl_dr_ts": f"sinks_demand_el_ts_hourly.csv",
    "demand_incl_dr_max": f"sinks_demand_el.csv",
    "demand_excl_dr_ts": f"sinks_demand_el_excl_demand_response_ts_{demand_response_scenario}_hourly.csv",
    "demand_excl_dr_max": f"sinks_demand_el_excl_demand_response_{demand_response_scenario}.csv",
    "dr_baseline": f"sinks_demand_response_el_ts_{demand_response_scenario}.csv",
    "dr_ava_pos": f"sinks_demand_response_el_ava_pos_ts_{demand_response_scenario}.csv",
    "dr_ava_neg": f"sinks_demand_response_el_ava_pos_ts_{demand_response_scenario}.csv",
}

DEMAND_RESPONSE_CLUSTERS = {
    "hoho_cluster_shift_only": "#111111", 
    "hoho_cluster_shift_shed": "#333333", 
    "ind_cluster_shed_only": "#555555",
    "ind_cluster_shift_only": "#999999", 
    "ind_cluster_shift_shed": "#aaaaaa", 
    "tcs+hoho_cluster_shift_only": "#cccccc",
    "tcs_cluster_shift_only": "#dddddd", 
}

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file_names = {
    "demand_incl_dr_ts": "demand_incl_dr_annual",
    "demand_excl_dr_ts": "demand_exl_dr_annual",
}

plt.rcParams.update({'font.size': 14})

## Read in and preprocess demand data

In [ ]:
data_sets = {
    data_set: pd.read_csv(f"{path_folder}{file_name}", index_col=0) 
    for data_set, file_name in file_names.items()
}

In [ ]:
# Slice 2020 data in the first place
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[f"{key}_DE_2020"] = data_sets[key].loc[
        data_sets[key].index.str[:4] == "2020", 
        [col for col in data_sets[key].columns if "DE_" in col]
    ]

## Combine to annual values

In [ ]:
map_ts_to_max = {
    "demand_incl_dr_ts": data_sets["demand_incl_dr_max"],
    "demand_excl_dr_ts": data_sets["demand_excl_dr_max"],
}

In [ ]:
# Create annual data sets
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[key].index = data_sets[key].index.str[:4]
    data_sets[f"{key}_annual"] = data_sets[key].groupby(data_sets[key].index).sum()["DE_sink_el_load"]
    data_sets[f"{key}_annual"] = data_sets[f"{key}_annual"].mul(
        map_ts_to_max[key].at["DE_sink_el_load", "maximum"]
    ).div(1000)
    data_sets[f"{key}_annual"].to_csv(f"{path_data_out}{output_file_names[key]}.csv")

In [ ]:
# Plot annual demand without demand response
fig, ax = plt.subplots(figsize=(12, 5))
_ = data_sets["demand_incl_dr_ts_annual"].loc["2020":"2045"].plot(kind="bar", ax=ax, color="darkblue")
_ = plt.xlabel("year")
_ = plt.ylabel("annual demand in GWh")
#_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_incl_dr_ts']}.png", dpi=300)
plt.show()
plt.close()

In [ ]:
data_sets["demand_excl_dr_ts_annual"]

In [ ]:
# Read in demand response potential parameter data and extract maximum value
dr_potential_data = {
    dr_cluster: pd.read_csv(
        f"{path_folder}{dr_cluster}_potential_parameters_{demand_response_scenario}%.csv", 
        index_col=0
    )
    for dr_cluster in DEMAND_RESPONSE_CLUSTERS
}
dr_max_potentials = {}
for key, val in dr_potential_data.items():
    dr_max_potentials[key] = val["max_cap"]

# Calculate annual demand (assumed constant here)
data_sets["dr_baseline"].index = data_sets["dr_baseline"].index.str[:4]

data_sets["dr_annual_demand"] = pd.DataFrame()
for dr_cluster in DEMAND_RESPONSE_CLUSTERS:
    data_sets["dr_annual_demand"][dr_cluster] = dr_max_potentials[dr_cluster].mul(
        data_sets["dr_baseline"].groupby(
            data_sets["dr_baseline"].index
        ).sum().at["2020", dr_cluster]
    ).div(1000)

data_sets["dr_annual_demand"].index = data_sets["dr_annual_demand"].index.astype(str)
    
# Combine with demand data set excluding demand response
data_sets["demand_excl_dr_ts_annual"] = pd.concat(
    [data_sets["demand_excl_dr_ts_annual"], data_sets["dr_annual_demand"]],
    axis=1
)
data_sets["demand_excl_dr_ts_annual"].rename(
    columns={"DE_sink_el_load": "load without demand response"},
    inplace=True
)
colors = {
    "load without demand response": "darkblue",
    **DEMAND_RESPONSE_CLUSTERS
}

In [ ]:
# Plot annual demand excl demand response with demand response baseline demand on top
fig, ax = plt.subplots(figsize=(12, 5))
data_sets["demand_excl_dr_ts_annual"] = data_sets["demand_excl_dr_ts_annual"][[col for col in colors]]
_ = data_sets["demand_excl_dr_ts_annual"].loc["2020":"2045"].plot(kind="bar", edgecolor="darkgrey", stacked=True, ax=ax, color=colors)
_ = plt.xlabel("year")
_ = plt.ylabel("annual demand in GWh")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_excl_dr_ts']}.png", dpi=300)
plt.show()
plt.close()

## Extract patterns

**TODO: Resume!**